## fill_clearing_and_imagingbatchsample_tables-dev
I decided to move the clearing batch and imaging batch information from the Sample() table and create new ClearingBatchSample() and ImagingBatchSample() tables to store that information. This becomes necessary when you have more than one imaging request so a sample could have multiple imaging batches for example.

In [1]:
import pickle, os
import os.path, sys
from datetime import datetime
import pandas as pd
import numpy as np
import datajoint as dj
dj.config["enable_python_native_blobs"] = True # So I can store python dictionaries in blob columns

## Connect to the db

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [3]:
# set up object for light sheet schema
db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
# db_lightsheet = dj.create_virtual_module('u19lightserv_lightsheet','u19lightserv_lightsheet')

## Fill ClearingBatchSample() and ImagingBatchSample() tables
After making these tables and removing the imaging and clearing batch information from the sample table, I need to make inserts for these tables from the pre-existing requests in the database

In [4]:
db_lightsheet.Request().ClearingBatchSample()

username user in the lab,request_name,sample_name,clearing_protocol,antibody1,antibody2,clearing_batch_number
lightserv-test,test_new_imaging_request,test_new_imaging_request-001,iDISCO abbreviated clearing,,,1
lightserv-test,test_new_imaging_request,test_new_imaging_request-002,iDISCO abbreviated clearing,,,1


In [5]:
db_lightsheet.Request().ClearingBatch()

username user in the lab,request_name,clearing_protocol,antibody1,antibody2,clearing_batch_number,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,None,None,complete,8,None,None,None,,https://docs.google.com/spreadsheets/d/1A83HVyy1bEhctqArwt4EiT637M8wBxTFodobbt1jrXI/edit#gid=0
ahoag,test,iDISCO abbreviated clearing,,,1,None,None,incomplete,1,None,None,ahoag,,None
ahoag,test123,iDISCO abbreviated clearing,,,1,None,None,complete,1,None,2020-01-01,ahoag,,None
ahoag,test_3p6x,iDISCO abbreviated clearing,,,1,None,None,complete,1,None,2020-01-01,ahoag,,None
ahoag,test_idiscoplus,iDISCO+_immuno,antibody 1 test,antibody 2 test,1,None,None,in progress,1,None,None,ahoag,,None
ahoag,test_large_smartspim,iDISCO abbreviated clearing,,,1,None,None,complete,1,None,None,ahoag,,None
ahoag,test_many_samples,iDISCO abbreviated clearing,,,1,None,None,in progress,10,2019-02-04,2020-01-01,ahoag,,None
ahoag,test_many_samples2,iDISCO abbreviated clearing,,,1,None,None,complete,4,None,None,ahoag,here are notes for clearer,None
ahoag,test_many_samples3,iDISCO abbreviated clearing,,,1,None,None,complete,3,None,None,ahoag,,None
ahoag,test_many_samples4,iDISCO abbreviated clearing,,,1,None,None,complete,2,None,None,ahoag,test notes,None


In [23]:
request_contents = db_lightsheet.Request().fetch(as_dict=True)
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()

imaging_request_contents = db_lightsheet.Request.ImagingRequest()
channel_contents = db_lightsheet.Request.ImagingChannel()

In [21]:
clearing_batch_sample_insert_list = []
# imaging_batch_sample_insert_list = []
for request_dict in request_contents[50:]:
    imaging_dict = {}
    username = request_dict['username']
    request_name = request_dict['request_name']
    print(f"Username: {username}")
    print(f"Request Name: {request_name}")
    print()
    date_submitted = request_dict['date_submitted']
    time_submitted = request_dict['time_submitted']
    clearing_batch_restrict_dict = {
        'username':username,
        'request_name':request_name,
        'clearing_batch_number':1,
    }
    # loop through samples and figure out clearing batches
    clearing_batch_contents_this_request = clearing_batch_contents & clearing_batch_restrict_dict
    sample_contents_this_request = (sample_contents & \
        {'username':username,'request_name':request_name}).fetch(as_dict=True)
    
    for sample_dict in sample_contents_this_request:
        sample_name = sample_dict['sample_name']
        clearing_batch_sample_dict = clearing_batch_restrict_dict.copy()
        clearing_batch_sample_dict['sample_name'] = sample_name
        clearing_protocol, antibody1, antibody2 = clearing_batch_contents_this_request.fetch1(
            'clearing_protocol','antibody1','antibody2')
        clearing_batch_sample_dict['clearing_protocol'] = clearing_protocol
        clearing_batch_sample_dict['antibody1'] = antibody1
        clearing_batch_sample_dict['antibody2'] = antibody2
        clearing_batch_sample_insert_list.append(clearing_batch_sample_dict)
   
# print(clearing_batch_sample_insert_list)
db_lightsheet.Request.ClearingBatchSample().insert(
    clearing_batch_sample_insert_list,skip_duplicates=True)
# imaging_dict

Username: lpinto
Request Name: sp4_20200106_albumin-FITC

Username: marlies.oostland
Request Name: 190809_M27

Username: marlies.oostland
Request Name: 20181217_marlies,_20181013_marlies_M12

Username: marlies.oostland
Request Name: M18,_M19,_M23,_M26,_M28,_M29,_M31

Username: marlies.oostland
Request Name: Marlies_190614_M21

Username: marlies.oostland
Request Name: Marlies_190618_M30

Username: marlies.oostland
Request Name: not_sure_-_check_when_samples_are_given_to_us?

Username: mioffe
Request Name: a1_20190327_a2_20190327_a3_20190327

Username: mioffe
Request Name: five_samples,_h1_and_h2,_r1,r2,r3

Username: mioffe
Request Name: M10,_M11

Username: pbibawi
Request Name: 20190405_pb_X015,_20190405_pb_X045,_20190405_pb_A233,20190405_pb

Username: pbibawi
Request Name: 20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019

Username: rmallarino
Request Name: 171

Username: soline
Request Name: 190725__Target_Practice

Username: soline
Request Name: Mouse092,_MouseK01

Use

In [28]:
imaging_batch_sample_insert_list = []
for request_dict in request_contents:
    is_archival = request_dict['is_archival']
    if not is_archival:
        continue
    imaging_dict = {}
    username = request_dict['username']
    request_name = request_dict['request_name']
    print(f"Username: {username}")
    print(f"Request Name: {request_name}")
    print()
    date_submitted = request_dict['date_submitted']
    time_submitted = request_dict['time_submitted']
    imaging_batch_restrict_dict = {
        'username':username,
        'request_name':request_name,
        'imaging_batch_number':1,
        'imaging_request_number':1
    }
    # loop through samples and figure out imaging batches
    imaging_batch_contents_this_request = imaging_batch_contents & imaging_batch_restrict_dict
    sample_contents_this_request = (sample_contents & \
        {'username':username,'request_name':request_name}).fetch(as_dict=True)
    
    for sample_dict in sample_contents_this_request:
        sample_name = sample_dict['sample_name']
        imaging_batch_sample_dict = imaging_batch_restrict_dict.copy()
        imaging_batch_sample_dict['sample_name'] = sample_name
        imaging_batch_sample_insert_list.append(imaging_batch_sample_dict)
   
print(imaging_batch_sample_insert_list)
db_lightsheet.Request.ImagingBatchSample().insert(
    imaging_batch_sample_insert_list,skip_duplicates=True)
# imaging_dict

Username: afalkner
Request Name: MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6

Username: apv2
Request Name: 20190313_IBL_DiI_1

Username: apv2
Request Name: ibl_witten_04

Username: ejdennis
Request Name: 10-13_brains,_names_TBD

Username: ejdennis
Request Name: 201905_atlas00x_where_x=1:n

Username: ejdennis
Request Name: 20190606_atlas00x_where_x=11-20

Username: ejdennis
Request Name: E112,_E126,_E137

Username: ejdennis
Request Name: K310_(CM-diI),_K315_(CM-diI),_K320,_K321,_K323,_K327,_K333,_K334

Username: ejdennis
Request Name: W118,_K292,_K293,_K295,_K301,_K302,_K303,_K304,_K305,_K306,_K307

Username: ejdennis
Request Name: W122,_W128,_E111

Username: ejdennis
Request Name: X013

Username: ejdennis
Request Name: X042

Username: ejdennis
Request Name: X050

Username: jverpeut
Request Name: 10032019_CNOtest

Username: jverpeut
Request Name: AdultChronicD_MLI_Lawrence_(1-12_each_batch)

Username: jverpeut
Request Name: an1-31

Username: jverpeut
Request Name: CDymaze_1-1